In [1]:
import pandas as pd
import requests

import re
pd.set_option('display.max_columns', None)


In [2]:
marineweb = "https://api.obis.org/v3/occurrence?scientificname=Neoturris%20pileata&size=100"
resmarineweb = requests.get(marineweb)


In [3]:
marineweb = "https://api.obis.org/v3/occurrence?scientificname=Neoturris%20pileata"
resmarineweb= requests.get(marineweb)

In [4]:
area = "https://api.obis.org/v3/area"
resarea= requests.get(area)

In [5]:
resarea.status_code

200

In [6]:
areadata = resarea.json()

In [7]:
dfarea = pd.DataFrame(areadata['results'])

In [8]:
areadata['results'][1].keys()

dict_keys(['id', 'name', 'type'])

In [9]:
dfarea.shape

(798, 3)

In [10]:
dfarea.head()

,id,name,type
0,10001,2les des Sept Frbres et Godorya (Seven Brother...,ebsa
1,1,ABNJ,abnj
2,6,ABNJ: Arctic Ocean,abnj
3,114,ABNJ: Indian Ocean,abnj
4,176,ABNJ: North Atlantic,abnj


In [11]:
especies = ["Neoturris%20pileata", "Megaptera%20novaeangliae","Phycocaris%20simulans", "Cheilinus%20undulatus", "Tursiops%20truncatus","Halichoerus%20grypus"]
dfs = []
columnas_por_especie = {}

In [12]:
for especie in especies:
    marineweb = f"https://api.obis.org/v3/occurrence?scientificname={especie}&size=100"
    
    try:
        resmarineweb = requests.get(marineweb)
        resmarineweb.raise_for_status()
        
        marinedata = resmarineweb.json()
        
        if marinedata.get("results"):
            # Definir las columnas deseadas
            columnas_deseadas = [
                "occurrenceID", "decimalLatitude", "decimalLongitude", "vernacularName", "scientificName", "basisOfRecord", "class", "subclass", 
                "family", "genus", "kingdom", "institutionCode", "species", "sex", "date_start", "date_end", 
                "date_year", "depth", "minimumDepthInMeters", "maximumDepthInMeters", 
                "organismQuantity", "organismQuantityType", "sampleSizeUnit", 
                "sampleSizeValue", "node_id", "absence", "flags", 
                "bathymetry", "shoredistance", "sst", "sss", "waterBody"
            ]
            
            # Obtener solo las columnas que existen en los datos
            df_temp = pd.DataFrame(marinedata["results"])
            columnas_existentes = list(set(columnas_deseadas).intersection(df_temp.columns))

            # Guardar información de columnas (sin imprimir todavía)
            columnas_por_especie[especie] = {
                'columnas_presentes': columnas_existentes,
                'columnas_faltantes': list(set(columnas_deseadas) - set(columnas_existentes))
            }
            
            # Crear DataFrame solo con las columnas disponibles
            df_especie = df_temp[columnas_existentes]
            
            dfs.append(df_especie)
            print(f"Datos obtenidos para {especie}: {len(df_especie)} ocurrencias")
        else:
            print(f"No se encontraron resultados para {especie}")
            
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos para {especie}: {e}")
    except KeyError as e:
        print(f"Error en la estructura de datos para {especie}: {e}")

# Imprimir resultados de columnas faltantes SOLO UNA VEZ al final
print("\n--- RESUMEN DE COLUMNAS FALTANTES POR ESPECIE ---")
for especie, info in columnas_por_especie.items():
    if info['columnas_faltantes']:
        print(f"{especie}: Faltan {len(info['columnas_faltantes'])} columnas")
        print(f"   Faltantes: {info['columnas_faltantes']}")

Datos obtenidos para Neoturris%20pileata: 100 ocurrencias
Datos obtenidos para Megaptera%20novaeangliae: 100 ocurrencias
Datos obtenidos para Phycocaris%20simulans: 19 ocurrencias
Datos obtenidos para Cheilinus%20undulatus: 100 ocurrencias
Datos obtenidos para Tursiops%20truncatus: 100 ocurrencias
Datos obtenidos para Halichoerus%20grypus: 100 ocurrencias

--- RESUMEN DE COLUMNAS FALTANTES POR ESPECIE ---
Neoturris%20pileata: Faltan 2 columnas
   Faltantes: ['waterBody', 'vernacularName']
Phycocaris%20simulans: Faltan 8 columnas
   Faltantes: ['maximumDepthInMeters', 'sex', 'sampleSizeUnit', 'waterBody', 'depth', 'vernacularName', 'minimumDepthInMeters', 'sampleSizeValue']
Cheilinus%20undulatus: Faltan 2 columnas
   Faltantes: ['subclass', 'sex']
Tursiops%20truncatus: Faltan 3 columnas
   Faltantes: ['sex', 'sampleSizeUnit', 'sampleSizeValue']
Halichoerus%20grypus: Faltan 4 columnas
   Faltantes: ['organismQuantity', 'sampleSizeValue', 'organismQuantityType', 'sampleSizeUnit']


In [13]:
marinedata["results"][0].keys()

dict_keys(['associatedReferences', 'basisOfRecord', 'bibliographicCitation', 'catalogNumber', 'class', 'classid', 'collectionCode', 'coordinatePrecision', 'coordinateUncertaintyInMeters', 'datasetID', 'datasetName', 'dateIdentified', 'date_end', 'date_mid', 'date_start', 'date_year', 'decimalLatitude', 'decimalLongitude', 'eventDate', 'eventTime', 'family', 'familyid', 'footprintWKT', 'genus', 'genusid', 'geodeticDatum', 'georeferencedDate', 'identificationRemarks', 'individualCount', 'infraorder', 'infraorderid', 'infraphylum', 'infraphylumid', 'institutionCode', 'kingdom', 'kingdomid', 'license', 'marine', 'megaclass', 'megaclassid', 'modified', 'nomenclaturalCode', 'occurrenceID', 'occurrenceRemarks', 'occurrenceStatus', 'order', 'orderid', 'organismID', 'organismRemarks', 'ownerInstitutionCode', 'phylum', 'phylumid', 'recordNumber', 'scientificName', 'scientificNameAuthorship', 'scientificNameID', 'sex', 'species', 'speciesid', 'specificEpithet', 'subclass', 'subclassid', 'suborder

In [14]:
epiturras = pd.concat(dfs, ignore_index=True)

In [15]:
epiturras["institutionCode"].value_counts()

institutionCode
CEBC                                                                                    79
Happywhale.com                                                                          70
SeaWatchFoundation                                                                      31
USDOC/NOAA/NMFS/PIFSC/ESD                                                               24
AIMS                                                                                    23
CRED                                                                                    21
MAGNT                                                                                    9
Observatoire Océanologique de Villefranche sur Mer                                       9
SPC                                                                                      8
CMLRE                                                                                    5
Diveboard                                                                 

In [23]:
epiturras.sample(10)

,date_year,sampleSizeUnit,class,scientificName,depth,institutionCode,shoredistance,decimalLatitude,flags,decimalLongitude,maximumDepthInMeters,absence,date_start,minimumDepthInMeters,subclass,basisOfRecord,sss,sex,organismQuantity,sst,family,kingdom,occurrenceID,bathymetry,node_id,date_end,organismQuantityType,species,sampleSizeValue,genus,waterBody,vernacularName
17,2002.0,DNA reads,Hydrozoa,Neoturris pileata,10.0,NaN,6836,50.250000,[],-4.217000,10.0,False,1.024877e+12,10.0,Hydroidolina,MaterialSample,34.79,NaN,0,12.86,Pandeidae,Animalia,584779,51.0,[f92d5d7f-47a6-4605-9fd0-a8538dfde3fd],1.024877e+12,DNA sequence reads,Neoturris pileata,40488,Neoturris,NaN,NaN
382,1979.0,NaN,Mammalia,Tursiops truncatus,NaN,University of Rhode Island,455,36.133000,[NO_DEPTH],-75.750000,NaN,False,2.971296e+11,NaN,Theria,HumanObservation,33.13,NaN,NaN,17.78,Delphinidae,Animalia,283_20130,-4.0,[573654c1-4ce7-4ea2-b2f1-e4d42f8f9c31],2.971296e+11,NaN,Tursiops truncatus,NaN,Tursiops,North Atlantic Ocean,Common Bottlenose Dolphin
323,2016.0,NaN,Mammalia,Tursiops truncatus,0.0,SeaWatchFoundation,353,52.218450,[],-4.352500,0.0,False,1.470010e+12,0.0,Theria,HumanObservation,34.04,NaN,NaN,12.01,Delphinidae,Animalia,SWF_13_225016_1,3.2,[f92d5d7f-47a6-4605-9fd0-a8538dfde3fd],1.470010e+12,NaN,Tursiops truncatus,NaN,Tursiops,NaN,Bottlenose dolphin
451,2012.0,NaN,Mammalia,Halichoerus grypus,NaN,CEBC,4998,48.601380,[NO_DEPTH],-4.803310,NaN,False,1.346717e+12,NaN,Theria,MachineObservation,35.16,Male,NaN,12.86,Phocidae,Animalia,2028_90567,90.0,[573654c1-4ce7-4ea2-b2f1-e4d42f8f9c31],1.346717e+12,NaN,Halichoerus grypus,NaN,Halichoerus,"Molene Archipelago,Iroise Sea,English Channel,...",Atlantic gray seal
73,2007.0,DNA reads,Hydrozoa,Neoturris pileata,10.0,NaN,6836,50.250000,[],-4.217000,10.0,False,1.172534e+12,10.0,Hydroidolina,MaterialSample,34.79,NaN,0,12.86,Pandeidae,Animalia,175126,51.0,[f92d5d7f-47a6-4605-9fd0-a8538dfde3fd],1.172534e+12,DNA sequence reads,Neoturris pileata,31551,Neoturris,NaN,NaN
108,1993.0,NaN,Mammalia,Megaptera novaeangliae,NaN,Happywhale.com,6729,37.643333,[NO_DEPTH],-123.048333,NaN,False,7.488288e+11,NaN,Theria,HumanObservation,32.41,NaN,NaN,12.39,Balaenopteridae,Animalia,251327,122.0,[573654c1-4ce7-4ea2-b2f1-e4d42f8f9c31],7.488288e+11,NaN,Megaptera novaeangliae,NaN,Megaptera,North Pacific Ocean,Humpback whale
34,2015.0,DNA reads,Hydrozoa,Neoturris pileata,10.0,NaN,6836,50.250000,[],-4.217000,10.0,False,1.423613e+12,10.0,Hydroidolina,MaterialSample,34.79,NaN,0,12.86,Pandeidae,Animalia,715393,51.0,[f92d5d7f-47a6-4605-9fd0-a8538dfde3fd],1.423613e+12,DNA sequence reads,Neoturris pileata,19434,Neoturris,NaN,NaN
8,2015.0,DNA reads,Hydrozoa,Neoturris pileata,10.0,NaN,6836,50.250000,[],-4.217000,10.0,False,1.421712e+12,10.0,Hydroidolina,MaterialSample,34.79,NaN,0,12.86,Pandeidae,Animalia,709456,51.0,[f92d5d7f-47a6-4605-9fd0-a8538dfde3fd],1.421712e+12,DNA sequence reads,Neoturris pileata,36756,Neoturris,NaN,NaN
358,1996.0,NaN,Mammalia,Tursiops truncatus,NaN,"The Dolphin Project, Inc.",-6957,31.775000,"[NO_DEPTH, ON_LAND]",-81.176700,NaN,False,8.293536e+11,NaN,Theria,HumanObservation,NaN,NaN,NaN,NaN,Delphinidae,Animalia,304_2486,-1.0,[573654c1-4ce7-4ea2-b2f1-e4d42f8f9c31],8.293536e+11,NaN,Tursiops truncatus,NaN,Tursiops,"Atlantic,Atlantic Ocean,East coast,estuarine w...",Common Bottlenose Dolphin
340,1992.0,NaN,Mammalia,Tursiops truncatus,0.0,NaN,2665,52.242222,[],-4.377500,0.0,False,7.211808e+11,0.0,Theria,HumanObservation,34.04,NaN,NaN,12.01,Delphinidae,Animalia,SWF_13_193599_1,25.6,[4bf79a01-65a9-4db6-b37b-18434f26ddfc],7.211808e+11,NaN,Tursiops truncatus,NaN,Tursiops,NaN,NaN


In [17]:
epiturras.shape

(519, 32)

In [18]:
epiturras.columns

Index(['date_year', 'sampleSizeUnit', 'class', 'scientificName', 'depth',
       'institutionCode', 'shoredistance', 'decimalLatitude', 'flags',
       'decimalLongitude', 'maximumDepthInMeters', 'absence', 'date_start',
       'minimumDepthInMeters', 'subclass', 'basisOfRecord', 'sss', 'sex',
       'organismQuantity', 'sst', 'family', 'kingdom', 'occurrenceID',
       'bathymetry', 'node_id', 'date_end', 'organismQuantityType', 'species',
       'sampleSizeValue', 'genus', 'waterBody', 'vernacularName'],
      dtype='object')

In [19]:
#milisegundos desde el 1 de enero de 1970
#medida Unix

In [20]:
epiturras['date_start'].value_counts()

date_start
-1.688342e+12    5
-1.704154e+12    3
 1.270685e+12    2
 1.378512e+12    2
 1.320624e+12    2
                ..
 1.682381e+12    1
 1.630973e+12    1
 1.719619e+12    1
 2.996352e+11    1
 1.307146e+12    1
Name: count, Length: 489, dtype: int64

In [21]:
epiturras['date_year'].value_counts()

date_year
2012.0    49
2011.0    36
2013.0    35
2023.0    24
2015.0    24
2009.0    19
2016.0    19
2006.0    18
2018.0    18
2022.0    17
2004.0    17
2024.0    16
2014.0    16
2010.0    16
2017.0    14
2007.0    14
2008.0    14
2002.0    11
2021.0    11
2020.0    10
2001.0    10
2003.0     9
2005.0     9
1916.0     9
1993.0     7
1991.0     6
1998.0     5
1979.0     4
1982.0     4
1995.0     4
1992.0     4
2000.0     3
2019.0     3
1899.0     3
1997.0     3
1985.0     2
1996.0     2
1958.0     2
1959.0     2
1999.0     2
2025.0     2
1960.0     2
1990.0     2
1984.0     2
1986.0     2
1981.0     2
1908.0     1
1954.0     1
1987.0     1
1822.0     1
1911.0     1
1896.0     1
1930.0     1
1910.0     1
1901.0     1
1912.0     1
1994.0     1
Name: count, dtype: int64

In [22]:
epiturras['dropped'].value_counts()

KeyError: 'dropped'

In [ ]:
epiturras.sample(5)

,date_end,minimumDepthInMeters,shoredistance,bathymetry,occurrenceID,sst,sampleSizeValue,organismQuantity,kingdom,sampleSizeUnit,scientificName,subclass,flags,class,organismQuantityType,sss,node_id,maximumDepthInMeters,date_year,depth,family,absence,genus,basisOfRecord,date_start,dropped,species
360,1.659398e+12,0.0,384,3.0,SWF_13_55082_3,10.19,NaN,NaN,Animalia,NaN,Tursiops truncatus,Theria,[],Mammalia,NaN,34.39,[f92d5d7f-47a6-4605-9fd0-a8538dfde3fd],0.0,2022.0,0.0,Delphinidae,False,Tursiops,HumanObservation,1.659398e+12,False,Tursiops truncatus
452,7.440768e+11,NaN,4066,10.0,71_234,9.83,NaN,NaN,Animalia,NaN,Halichoerus grypus,Theria,[NO_DEPTH],Mammalia,NaN,33.78,[573654c1-4ce7-4ea2-b2f1-e4d42f8f9c31],NaN,1993.0,NaN,Phocidae,False,Halichoerus,MachineObservation,7.440768e+11,False,Halichoerus grypus
354,1.462925e+12,0.0,2367,117.0,58691431-732b-4382-9b43-6553d4fc95a1,19.08,NaN,1,Animalia,NaN,Tursiops truncatus,Theria,[],Mammalia,individuals,36.24,[4bf79a01-65a9-4db6-b37b-18434f26ddfc],0.0,2016.0,0.0,Delphinidae,False,Tursiops,HumanObservation,1.462925e+12,False,Tursiops truncatus
433,1.381882e+12,NaN,4174,78.4,2028_112699,12.94,NaN,NaN,Animalia,NaN,Halichoerus grypus,Theria,[NO_DEPTH],Mammalia,NaN,35.15,[573654c1-4ce7-4ea2-b2f1-e4d42f8f9c31],NaN,2013.0,NaN,Phocidae,False,Halichoerus,MachineObservation,1.381882e+12,False,Halichoerus grypus
139,1.645574e+12,NaN,3800,50.0,291222,25.38,NaN,NaN,Animalia,NaN,Megaptera novaeangliae,Theria,[NO_DEPTH],Mammalia,NaN,34.99,[573654c1-4ce7-4ea2-b2f1-e4d42f8f9c31],NaN,2022.0,NaN,Balaenopteridae,False,Megaptera,HumanObservation,1.645574e+12,False,Megaptera novaeangliae


In [ ]:
epiturras.to_csv("marinedata.csv")

In [ ]:
#guardar el df a csv